In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Activation:
    def activation(self, x):
        pass
    
    def derivate(self, x):
        pass

In [3]:
class Tanh(Activation):
    def activation(self, x):
        return np.tanh(x)
    
    def prime(self, x):
        return 1 - np.tanh(x) ** 2 

In [4]:
class Sigmoide(Activation):
    def activation(self, x):
        return 1 / (1 + np.exp(-x))
    
    def prime(self, x):
        sigmoide = self.activation(x)
        return sigmoide * (1 - sigmoide)

In [ ]:
class ReLu(Activation):
    def activation(self, x):
        return np.maximum(0, x)
    
    def prime(self, x):
        return x > 0

In [ ]:
class MLP:
    def __init__(self, activation):
        self.activation = activation()
    
    def backpropagation(self):
        pass